# MIS: Clase 15 (2023-20)

El objetivo de esta clase es explicar los algoritmos que se pidió implementar la clase pasada.

#### Preliminares.

In [1]:
# Cargue de paquetes
import numpy as np
import random
import copy

In [2]:
# Generador de agentes
def agentes(n:int,restricciones:list=1):
    if isinstance(restricciones,int):
        A = {"A_{}".format(i):restricciones for i in range(n)}
    elif isinstance(restricciones,list):
        if len(restricciones)==n:
            A = {"A_{}".format(i):restricciones[i] for i in range(n)}
        else:
            raise Exception("Introduzca Parámetros Adecuados") 
    return A
    
# Generador de objetos
def objetos(m:int):
    return ["O_{}".format(i) for i in range(m)]

# Generador de preferencias
def preferencias(agentes:dict,objetos:list):
    pref = {}
    for agente in agentes:
        obj = copy.deepcopy(objetos)
        cont = 0
        n=len(obj)
        arreglo = {}
        arreglo1 = {}
        while 0<n:
            i = random.randint(0,n-1)
            U = obj.pop(i)
            arreglo[cont] = U
            arreglo1[U]=cont
            n -= 1
            cont += 1
        pref[agente] = (arreglo,arreglo1,agentes[agente]) #Retorna la relación Objeto-Posición, Posición-Objeto y la cantidad de objetos que puede tener el agente
    return pref

In [3]:
# Ejemplos de Prueba
Agentes = agentes(3,2)
Objetos = objetos(5)
Preferencias = preferencias(Agentes,Objetos)

#### Dictadores Seriales 1:1 y m:1.

In [4]:
def Dict_Serial_1_1(Agentes,Objetos,Preferencias,aleatorio=False):
    AAA = list(Agentes.keys()) #Obtengo Agentes
    if aleatorio:
        random.shuffle(AAA) # Hago aleatorio el orden de elección
    Asignaciones = {obj:False for obj in Objetos} # Inicializo las asiganciones
    for pos in range(len(AAA)): # Recorro las posiciones de los agentes
        agent = AAA[pos] # Obtengo el agente que va en este turno
        Pref = Preferencias[agent][0] # Obtengo las preferencias de los agentes
        for j in Pref: # Recorro el orden de preferencias 
            Obj = Pref[j]
            if Asignaciones[Obj]==False:
                Asignaciones[Obj]=agent
                break
    return Asignaciones # Retorno asignaciones

In [5]:
def Dict_Serial_m_1(Agentes,Objetos,Preferencias,aleatorio=False):
    AAA = list(Agentes.keys()) #Obtengo Agentes
    if aleatorio:
        random.shuffle(AAA) # Hago aleatorio el orden de elección
    Ag = copy.deepcopy(Agentes)
    Asignaciones = {obj:False for obj in Objetos} # Inicializo las asiganciones
    for pos in range(len(AAA)): # Recorro las posiciones de los agentes
        agent = AAA[pos] # Obtengo el agente que va en este turno
        Pref = Preferencias[agent][0] # Obtengo las preferencias de los agentes
        for j in Pref: # Recorro el orden de preferencias 
            if Ag[agent]>0:# Reviso si es posible hacer una asignación
                Obj = Pref[j] # Tomo el objeto
                if Asignaciones[Obj]==False: # Reviso si no está asignado
                    Asignaciones[Obj]=agent # Asigno
                    Ag[agent] = Ag[agent]-1 # Actualizo el cupo de objetos que el agente tiene disponible
            else:
                break # Dejo de buscar asignaciones para este agente si ya tiene su cupo completo
    return Asignaciones # Retorno asignaciones

In [6]:
Dict_Serial_1_1(Agentes,Objetos,Preferencias)

{'O_0': False, 'O_1': False, 'O_2': 'A_1', 'O_3': 'A_0', 'O_4': 'A_2'}

In [7]:
Dict_Serial_m_1(Agentes,Objetos,Preferencias)


{'O_0': 'A_1', 'O_1': 'A_2', 'O_2': 'A_1', 'O_3': 'A_0', 'O_4': 'A_0'}

#### Dictadores Seriales Inversos 1:1 y m:1. (Ley de la Selva).

In [8]:
# Para estos modelos, hay que hacer unas funciones auxiliares antes.
def llave_dado_valor(diccionario,valor):
    key = list(filter(lambda x: diccionario[x] == valor, diccionario))[0]
    return key

def asignador_forzoso(Preferencias,Agentes,agente_despojado,asignaciones,orden_fuerza):
    pos = llave_dado_valor(orden_fuerza,agente_despojado) #Encuentro la fuerza del agente despojado
    Agentes[agente_despojado] = Agentes[agente_despojado]+1 # Hago evidente que al haberle quitado un objeto al agente, su cupo incrementa en uno
    Pref = Preferencias[agente_despojado][0] # Obtengo las preferencias del agente despojado
    for j in Pref: # Recorro el orden de preferencias 
        Obj = Pref[j] # Tomo el objeto a asignar
        if asignaciones[Obj]==False: # Si no ha sido asignado, asigno
            asignaciones[Obj]=agente_despojado # Asigno
            Agentes[agente_despojado] = Agentes[agente_despojado]-1 # Actualizo el cupo de objetos del agente
            break
        elif pos<llave_dado_valor(orden_fuerza,asignaciones[Obj]): # Reviso si el agente puede usar fuerza
            agente_despojado1 = asignaciones[Obj] # Identifico al agente que le voy a quitar su objeto
            asignaciones[Obj] = agente_despojado # Asigno el objeto al agente
            Agentes[agente_despojado] = Agentes[agente_despojado]-1 # Actualizo el cupo del agente
            asignaciones,Agentes = asignador_forzoso(Preferencias,Agentes,agente_despojado1,asignaciones,orden_fuerza) # Realizo el proceso de asignación al agente despojado
            break
    return asignaciones,Agentes

In [10]:
def Dict_Serial_Inverso_1_1(Agentes,Objetos,Preferencias,aleatorio=False):
    AAA = list(Agentes.keys()) #Obtengo Agentes
    if aleatorio:
        random.shuffle(AAA) # Hago aleatorio el orden de elección
    AAA = {i:AAA[i] for i in range(len(AAA))} # Hago explicita la relación agente-orden(fuerza)
    Ag = copy.deepcopy(Agentes)
    Asignaciones = {obj:False for obj in Objetos} # Inicializo las asiganciones
    pos = len(AAA)-1 # Inicializo la posición final, i.e. inicio por el agente más débil.
    while pos>=0: # Recorro las posiciones de los agentes
        agent = AAA[pos] # Obtengo el agente que va en este turno
        Pref = Preferencias[agent][0] # Obtengo las preferencias de los agentes
        for j in Pref: # Recorro el orden de preferencias 
            Obj = Pref[j] # Tomo el objeto a asignar
            if Asignaciones[Obj]==False: # Si no ha sido asignado, asigno
                Asignaciones[Obj]=agent
                break
            elif llave_dado_valor(AAA,agent)<llave_dado_valor(AAA,Asignaciones[Obj]): # Reviso si el agente puede usar fuerza (piense por qué esta condición siempre se cumple)
                agent_despojado = Asignaciones[Obj] # Identifico al agente que le voy a quitar su objeto
                Asignaciones[Obj]=agent # Asigno el objeto al agente
                Asignaciones,Ag = asignador_forzoso(Preferencias,Ag,agent_despojado,Asignaciones,AAA)# Realizo el proceso de asignación al agente despojado
                break
        pos-=1 # Paso al siguiente agente en la lista
    return Asignaciones # Retorno asignaciones

In [11]:
def Dict_Serial_Inverso_m_1(Agentes,Objetos,Preferencias,aleatorio=False):
    AAA = list(Agentes.keys()) #Obtengo Agentes
    if aleatorio:
        random.shuffle(AAA) # Hago aleatorio el orden de elección
    AAA = {i:AAA[i] for i in range(len(AAA))} # Hago explicita la relación agente-orden(fuerza)
    Ag = copy.deepcopy(Agentes)
    Asignaciones = {obj:False for obj in Objetos} # Inicializo las asiganciones
    pos = len(AAA)-1 # Inicializo la posición final, i.e. inicio por el agente más débil.
    while pos>=0: # Recorro las posiciones de los agentes
        agent = AAA[pos] # Obtengo el agente que va en este turno
        Pref = Preferencias[agent][0] # Obtengo las preferencias de los agentes
        for j in Pref: # Recorro el orden de preferencias 
            if Ag[agent]>0: # Reviso si es posible hacer asignación al agente
                Obj = Pref[j] # Tomo el objeto a asignar
                if Asignaciones[Obj]==False: # Si no ha sido asignado, asigno
                    Asignaciones[Obj]=agent # Asigno
                    Ag[agent] = Ag[agent]-1 # Actualizo el cupo de objetos del agente
                elif llave_dado_valor(AAA,agent)<llave_dado_valor(AAA,Asignaciones[Obj]): # Reviso si el agente puede usar fuerza (piense por qué esta condición siempre se cumple)
                    agent_despojado = Asignaciones[Obj] # Identifico al agente que le voy a quitar su objeto
                    Asignaciones[Obj]=agent # Asigno el objeto al agente
                    Ag[agent] = Ag[agent]-1 # Actualizo el cupo de objetos del agente
                    Asignaciones,Ag = asignador_forzoso(Preferencias,Ag,agent_despojado,Asignaciones,AAA)# Realizo el proceso de asignación al agente despojado
            else:
                break # Dejo de buscar asignaciones para este agente.
        pos-=1 # Paso al siguiente agente en la lista
    return Asignaciones # Retorno asignaciones

In [12]:
Dict_Serial_Inverso_1_1(Agentes,Objetos,Preferencias)

{'O_0': False, 'O_1': False, 'O_2': 'A_1', 'O_3': 'A_0', 'O_4': 'A_2'}

In [13]:
Dict_Serial_Inverso_m_1(Agentes,Objetos,Preferencias)

{'O_0': 'A_1', 'O_1': 'A_2', 'O_2': 'A_1', 'O_3': 'A_0', 'O_4': 'A_0'}

#### Tarea:

1. Entienda por qué Dictador Serial y Dictador Serial inverso (1:1 y m:1) tienen los mismos resultados

2. Piense una forma (no computacionalmente eficiente), pero sencilla de implementar en código el algoritmo de TTC.